## This document contains commands we used to run different tools in our publication.

#### Metagenomic sample pre-processing

In [3]:
## Trimgalore!

# trim_galore --paired <fastq1> <fastq2> -> this command outputs post-QC fastq files

## Human contamination removal with bowtie2

# wget https://genome-idx.s3.amazonaws.com/bt/GRCh38_noalt_as.zip
# unzip GRCh38_noalt_as.zip
# bowtie2 -p 8 -x GRCh38_noalt_as -1 <fastq1> -2 <fastq2> --very-sensitive-local --un-conc-gz SAMPLE_host_removed > SAMPLE_mapped_and_unmapped.sam
# mv SAMPLE_host_removed.1 SAMPLE_host_removed_R1.fastq.gz
# mv SAMPLE_host_removed.2 SAMPLE_host_removed_R2.fastq.gz

## Taxonomic assignment with MetaPhlAn

# zcat SAMPLE_host_removed_R1.fastq.gz SAMPLE_host_removed_R2.fastq.gz | gzip -c > metagenome.fastq
# metaphlan metagenome.fastq --input_type fastq -o profiled_metagenome.txt

## Functional prediction with Humann

# humann --input metagenome.fastq --output output_metagenome

#### Running different indices

In [ ]:
## Shannon entropy - on species and functions

#biom convert -i <profile txt> -o profile.biom --table-type "Table" --to-hdf5
#qiime tools import --input-path profile.biom --type FeatureTable[Frequency] --output-path profile.qza
#qiime diversity alpha --i-table profile.qza --p-metric shannon --o-alpha-diversity profile_shannon.qza
#qiime diversity alpha-group-significance --i-alpha-diversity profile_shannon.qza --m-metadata-file metadata.txt --o-visualization profile_shannon.qzv

## GMHI

#biom convert -i <profile txt> -o profile.biom --table-type "Table" --to-hdf5
#qiime tools import --input-path profile.biom --type FeatureTable[Frequency] --output-path profile.qza
#qiime health-index gmhi-predict-viz --i-table profile.qza --m-metadata-file metadata.txt --o-gmhi-results profile_results --o-gmhi-plot profile_plot

## hiPCA

# To calculate hiPCA scores, we modified the original MATLAB script by substituting inputs with our files and saved the output values.

## Q2PD

# python run_q2_predict_dysbiosis.py

#### Index benchmarking

In [ ]:
## Boruta - run on matrix where parameters are values of different indices and the decision variable is health/disease

#import pandas as pd
#from sklearn.ensemble import RandomForestClassifier
#from boruta import BorutaPy
#import numpy as np 
#from sklearn.metrics import roc_curve, roc_auc_score 

# BorutaPy accepts numpy arrays only, hence the .values attribute
#X = pd.read_csv('input.txt', sep="\t",usecols=[0,1,2,3]).values
#y = pd.read_csv('input.txt', sep="\t",usecols=[4]).values
#y = y.ravel()

#cols = list(pd.read_csv('input.txt', sep="\t",usecols=[0,1,2,3]).columns)

# define random forest classifier, with utilising all cores and
# sampling in proportion to y labels
#rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# define Boruta feature selection method
#feat_selector = BorutaPy(rf, n_estimators='auto', verbose=0, random_state=1)

# find all relevant features - 5 features should be selected
#feat_selector.fit(X, y)

#print("Ranking:")
#print(feat_selector.ranking_)

#print("\n------Support and Ranking for each feature------")
#for i in range(len(feat_selector.support_)):
#    if feat_selector.support_[i]:
#        print("Passes the test: ", cols[i],
#              " - Ranking: ", feat_selector.ranking_[i])
#    else:
#        print("Doesn't pass the test: ",
#              cols[i], " - Ranking: ", feat_selector.ranking_[i])        
        
#X_filtered = feat_selector.transform(np.array(X))
#rf.fit(X_filtered, y)
#predictions = rf.predict(X_filtered)

# create a dataframe with real predictions and values
#df = pd.DataFrame({'pred': predictions, 'observed': y})

# compute RMSE
#mse = ((df['pred'] - df['observed']) ** 2).mean()
#rmse = np.sqrt(mse)

# RF accuracy and AUC ROC
#roc_auc = roc_auc_score(y, predictions) 


#### Other tools used in the study

In [ ]:
## MDFS

#data_vector = X
#dec_vector = y
#dec_vector[dec_vector != 0] = 1
#output = mdfs.run(np.array(data_vector), np.array(dec_vector.astype(np.int32)), n_contrast=30, dimensions=2, discretizations=50, range_=0.9)

#all_labels = list(X.columns)
#important_labels = []
#for i in output['relevant_variables']:
#    important_labels.append(all_labels[i])
#print(important_labels)

In [ ]:
## SPARCC - separately per project and diagnosis

#biom convert -i profile.tsv -o profile.biom --table-type "Table" --to-hdf5
#qiime tools import --input-path profile.biom --type FeatureTable[Frequency] --output-path profile.qza

#qiime SCNIC calculate-correlations --i-table profile.qza --p-method sparcc --o-correlation-table profile_sparcc.qza
#qiime tools export --input-path profile_sparcc.qza --output-path profile_sparcc

# in python:

#sparcc_results = pd.read_csv("profile_sparcc/pairwise_comparisons.tsv", sep="\t")
#sparcc_results = sparcc_results.loc[sparcc_results['r'].abs() >= 0.1]

In [ ]:
## Random Forest training

#from numpy import mean
#from numpy import std
#from pandas import read_csv
#from sklearn.model_selection import LeaveOneOut
#from sklearn.model_selection import cross_val_predict
#from sklearn.ensemble import RandomForestClassifier


# create loocv procedure
#cv = LeaveOneOut()
# create model
#model = RandomForestClassifier(random_state=1)
# evaluate model
#scores = cross_val_predict(model, X, y, method='predict_proba', cv=cv, n_jobs=-1)
# report performance
#print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))